## Project 2: Thermoelectric Generator

Import and Install Libraries

In [80]:
%pip install pandas pyfluids sympy scipy numpy matplotlib openpyxl

import pandas as pd # This makes nice tables
import pyfluids # This does the thermodynamics
import math
import scipy.optimize
from sympy import symbols, Eq, solve
import numpy as np
from scipy.optimize import fsolve
from scipy.optimize import root
import matplotlib.pyplot as plt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Power Requirements

In [81]:
power_data = [
    ["Raspberry Pi Pico", 42, 0.1386],
    ["Lora Radio Module", 125, 0.4125],
    ["pH Sensor", 13.3, 0.04389],
    ["Salinity", 34, 0.1122],
]
power_draw = pd.DataFrame(power_data, columns=["Component", "Peak Amperage Draw (mA)", "Power Draw (W)"])

power_draw

,Component,Peak Amperage Draw (mA),Power Draw (W)
0,Raspberry Pi Pico,42.0,0.13860
1,Lora Radio Module,125.0,0.41250
2,pH Sensor,13.3,0.04389
3,Salinity,34.0,0.11220


Temperature Data

In [82]:
minT_data = [
    [1,24],
    [2,25],
    [3,26],
    [4,21],
]
min_Temp = pd.DataFrame(minT_data, columns=["Month", "Min Average Temp (C)"])

min_Temp

,Month,Min Average Temp (C)
0,1,24
1,2,25
2,3,26
3,4,21


In [83]:
maxT_data = [
    [1,32],
    [2,36],
    [3,37],
    [4,32],
]
max_Temp = pd.DataFrame(maxT_data, columns=["Month", "Max Average Temp (C)"])

max_Temp

,Month,Max Average Temp (C)
0,1,32
1,2,36
2,3,37
3,4,32


Solar Irradiation

In [84]:
irradiation_data = [
    ["Min",4.38],
    ["Max",4.93],
]
irradiation = pd.DataFrame(irradiation_data, columns=["Solar Irradiation", "Daily Output (kWh/m^2)"])

irradiation

,Solar Irradiation,Daily Output (kWh/m^2)
0,Min,4.38
1,Max,4.93


Wind Speed

In [85]:
wind_Data = [[3]]
wind_Speed = pd.DataFrame(wind_Data,columns = ["Average Wind Speed (m/s)"])
wind_Speed

,Average Wind Speed (m/s)
0,3


Hourly Constants

In [86]:
# Load the Excel file into a pandas DataFrame
constant_frame = pd.read_excel('HourlyConstants.xlsx', header=1)

print(constant_frame.columns)

# Now you can work with the DataFrame 'df'
print(constant_frame.head())  # Display the first few rows of the DataFrame?

FileNotFoundError: [Errno 2] No such file or directory: 'HourlyConstants.xlsx'

Heat Transfer Characteristics

In [93]:
heat_data = [
    ["Number of Units",21,"3 sets of 7 in series"],
    ["Unit Length",0.03,"m"],
    ["Unit Thickness",3.75E-3,"m"],
    ["Thermal Conductivity",1.72,"W/mK"],
    ["Emissivity",.9,"Painted Black"],
    ["Absorptivity",.9,"Painted Black"],
    ["Assumed Water Temperature",30,"Degree C"],
]
heat_parameters = pd.DataFrame(heat_data, columns=["Parameter", "Value","Units/Notes"])

heat_parameters

,Parameter,Value,Units/Notes
0,Number of Units,21.00000,3 sets of 7 in series
1,Unit Length,0.03000,m
2,Unit Thickness,0.00375,m
3,Thermal Conductivity,1.72000,W/mK
4,Emissivity,0.90000,Painted Black
5,Absorptivity,0.90000,Painted Black
6,Assumed Water Temperature,30.00000,Degree C


In [94]:
from pyfluids import Fluid, FluidsList, Input

water = Fluid(FluidsList.Water)

water_ambient = water.with_state(
    Input.pressure(101325), Input.temperature(30)
)
#print(water_ambient.density)

Natural Convection for the Bottom of the Plate

In [95]:
def calcBeta(Tc,Tw):
    """
    Calculates Beta coefficient

    Parameters:
    -Tc (float): Temp (K) of the cold side of the thermoelectric generator
    -Th (float): Temp (K) of the hot side of the thermoelectric generator

    Returns:
    -beta (float): coefficient value
    """
    water_Amb = water.with_state(
    Input.pressure(101325), Input.temperature(Tw-273)
    )

    rhoAmb= water_Amb.density

    T = Tc - 273
    rho = (999.83952 + 16.945176 * T - 7.9870401E-3 * (T**2) - 49.170461E-6 * (T**3) + 105.56302E-9 * (T**4) - 280.54253E-12 * (T**5))/(1+16.897850E-3 * T)
    print(rho)

    beta = -1/rho * (rhoAmb-rho)/(Tw-Tc)
    return beta


In [96]:
def calcRalyeighNumber(beta, Tc, Tw, x, alpha):
    """
    Calculates RalyeighNumber

    Parameters:
    -Tc (float): Temp (K) of the cold side of the thermoelectric generator
    -beta (float): coefficient value
    -Tw (float): Temp (K) of the water
    -x (float): Length in meters
    -alpha (float)

    Returns:
    -RaX
    """
    #Find dT
    deltaT = Tc - Tw
    #Define Constants
    g = 9.81
    kinematic_viscosity = 8.005E-7
    #Calculate Ra
    Rax  = abs(g * beta * deltaT * (x)**3 / (kinematic_viscosity * alpha))
    return Rax

In [97]:
def bottomConvection(Tc,Tw,L,alpha):

    Pr = 6.9
    beta = calcBeta(Tc,Tw)
    
    #Calculate Ralyeigh Number at x = L
    RaL = calcRalyeighNumber(beta,Tc, Tw, L, alpha)

    #Critical Rayleigh Number
    if RaL < 1E9:
        NuL = 0.68 + 0.670 * (RaL) ** (1/4) / (( 1- (0.492/Pr) ** (9/16)) ** (4/9))
    else:
        NuL = (0.825+ 0.397 * (RaL) ** (1/6) / (( 1 + (0.492/Pr) ** (9/16)) ** (8/27))**2)

    kf = 0.598E-3 # kW/mK


    return NuL



Forced Convection on Top of the Plate

In [98]:
def topForcedConvectionCoefficient(u,Lplate):
    """
    Calculates h for the top plate due to wind

    Parameters:
    -u (float): Wind speed (m/s)
    -Lplate (float):Length of solar abosrber (m)

    Returns:
    -ha (float): Convection coefficient (W/m^2 K)
    """
    #Forced Convection on the top of the plate
    #Constants for 
    kf = 27.06 #W/(mk)
    kinematic_viscosity = 1.61E-5
    Pr = 0.71

    Re = (u**2)  * Lplate / (kinematic_viscosity)

    NuL = 0.453 * (Re ** (1/2)) * (Pr ** 1/3)
    ha = NuL * kf / Lplate

    return ha


Heat Balance Equations

In [99]:
def updatedSolution(u, Lplate, Tw, L, alpha, G, epsilon, Tamb, Kt, N, t):
    """
    Numerically solves for Th (Temp (K) of hot side of thermoelectric generator), and Tc (Temp (K) of cold side of thermoelectric generator)

    Parameters:
    -u (float): Wind speed (m/s)
    -Lplate (float):Length of solar abosrber (m)
    -L (float): length of thermoelectric generator module (m)
    -alpha (float):
    -G (float): 
    -epsilon (float): emmisivity 
    -Tamb (float): Ambient Temperature (K)
    -Kt (float): Thermal conductivity of the thermoelectric element
    -N (int): number of modules
    -t (float): Thickness of the thermoelectric element (m)

    
    """    
     
    def residuals(vars):
        Th, Tc, hw, rho = vars

        ha = topForcedConvectionCoefficient(u, Lplate)
        sigma = 5.67E-8

        RHS = G*alpha - sigma * epsilon * (Th)**4 - ha * (Th-Tamb) 
        LHS = N * Kt * t * (Th-Tc)

        eq1 = RHS - LHS
        eq2 = N*Kt*t * (Th-Tc) - hw * (Tc-Tw)

        # Calculate rho
        water_Amb = water.with_state(Input.pressure(101325), Input.temperature(Tw-273))
        rhoAmb = water_Amb.density
        rho_expr = (999.83952 + 16.945176 * (Tc - 273) - 7.9870401E-3 * ((Tc - 273)**2) - 49.170461E-6 * ((Tc - 273)**3) + 105.56302E-9 * ((Tc - 273)**4) - 280.54253E-12 * ((Tc - 273)**5))/(1+16.897850E-3 * (Tc - 273))
        eqn4 = rho - rho_expr

        Beta = -1/rho * (rhoAmb-rho)/(Tw-Tc)

        # Calculate RaL
        deltaT = Tc - Tw
        g = 9.81
        kinematic_viscosity = 8.005E-7
        RaL = abs(g * Beta * deltaT * (L)**3 / (kinematic_viscosity * alpha))

        Pr = 6.9
        NuL = 0.68 + 0.670 * (RaL) ** (1/4) / (( 1- (0.492/Pr) ** (9/16)) ** (4/9))
        
        eqn3 = hw - NuL * 0.6 / L
        return [eq1, eq2, eqn3, eqn4]

    initial_guess = [350, 300, 50, 900]
    solution = root(residuals, initial_guess).x
    print(solution)
    

In [107]:
updatedSolution(0.5,0.5,(30+273), 30E-3, 0.9,800,0.9,(36+273),1.7,1,3E-3)

[309.49246317 303.00144008  22.98781352 995.23627113]
